In [56]:
import findspark
findspark.init('/home/realtour/sparkTutorial/spark-2.4.7-bin-hadoop2.7')


In [57]:
#notes
"""
how to install mysql jar
# https://dev.mysql.com/get/Downloads/Connector-J/mysql-connector-java_8.0.23-1ubuntu20.10_all.deb
download and run command
sudo dpkg -i mysql-connector-java_8.0.23-1ubuntu20.10_all.deb

after installation checj in cd /usr/share/java/mysql-connector-java-8.0.23.jar
"""

'\nhow to install mysql jar\n# https://dev.mysql.com/get/Downloads/Connector-J/mysql-connector-java_8.0.23-1ubuntu20.10_all.deb\ndownload and run command\nsudo dpkg -i mysql-connector-java_8.0.23-1ubuntu20.10_all.deb\n\nafter installation checj in cd /usr/share/java/mysql-connector-java-8.0.23.jar\n'

In [58]:
#create spark session
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
import pyspark # only run after findspark.init()

spark = SparkSession \
        .builder \
        .appName('test') \
        .master('local[*]') \
        .config("spark.driver.extraClassPath", "/usr/share/java/mysql-connector-java-8.0.23.jar") \
        .getOrCreate()

spark


In [59]:

employees_df = spark.read.format("jdbc").option("url","jdbc:mysql://localhost:3306/employees").option("driver","com.mysql.jdbc.Driver").option("dbtable","employees").option("user","root").option("password","Meta@123").load()
dept_emp_df = spark.read.format("jdbc").option("url","jdbc:mysql://localhost:3306/employees").option("driver","com.mysql.jdbc.Driver").option("dbtable","dept_emp").option("user","root").option("password","Meta@123").load()
departments_df = spark.read.format("jdbc").option("url","jdbc:mysql://localhost:3306/employees").option("driver","com.mysql.jdbc.Driver").option("dbtable","departments").option("user","root").option("password","Meta@123").load()
titles_df = spark.read.format("jdbc").option("url","jdbc:mysql://localhost:3306/employees").option("driver","com.mysql.jdbc.Driver").option("dbtable","titles").option("user","root").option("password","Meta@123").load()

In [60]:
#create sql view of this df
titles_df.createOrReplaceTempView("titles")
employees_df.createOrReplaceTempView("employees")
dept_emp_df.createOrReplaceTempView("dept_emp")
departments_df.createOrReplaceTempView("departments")

In [61]:
titles_df.printSchema()

root
 |-- emp_no: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- from_date: date (nullable = true)
 |-- to_date: date (nullable = true)



In [62]:
titles_df.limit(5).toPandas()


,emp_no,title,from_date,to_date
0,10001,Senior Engineer,1986-06-26,9999-01-01
1,10002,Staff,1996-08-03,9999-01-01
2,10003,Senior Engineer,1995-12-03,9999-01-01
3,10004,Engineer,1986-12-01,1995-12-01
4,10004,Senior Engineer,1995-12-01,9999-01-01


In [63]:
#let display the total record in this table
spark.sql("select count(*) as total_record from titles").show()

+------------+
|total_record|
+------------+
|      443308|
+------------+



In [64]:
spark.sql("select * from titles limit 5").show()

+------+---------------+----------+----------+
|emp_no|          title| from_date|   to_date|
+------+---------------+----------+----------+
| 10001|Senior Engineer|1986-06-26|9999-01-01|
| 10002|          Staff|1996-08-03|9999-01-01|
| 10003|Senior Engineer|1995-12-03|9999-01-01|
| 10004|       Engineer|1986-12-01|1995-12-01|
| 10004|Senior Engineer|1995-12-01|9999-01-01|
+------+---------------+----------+----------+



In [65]:
#lets find agg of title
spark.sql("select title,count(title) as total_emp from titles group by title order by total_emp desc").show()

+------------------+---------+
|             title|total_emp|
+------------------+---------+
|          Engineer|   115003|
|             Staff|   107391|
|   Senior Engineer|    97750|
|      Senior Staff|    92853|
|  Technique Leader|    15159|
|Assistant Engineer|    15128|
|           Manager|       24|
+------------------+---------+



In [66]:
#join two table and store data into postgres db
# select employees.emp_no,employees.first_name,employees.last_name,departments.dept_name from employees inner join dept_emp on employees.emp_no=dept_emp.emp_no inner join departments on dept_emp.dept_no=departments.dept_no order by employees.emp_no asc limit 5;
#Q:join three table and print employees associate their dept_name at asc order

In [67]:
sql_query="select employees.emp_no,employees.first_name,employees.last_name,departments.dept_name from employees inner join dept_emp on employees.emp_no=dept_emp.emp_no inner join departments on dept_emp.dept_no=departments.dept_no order by employees.emp_no asc"
result_df=spark.sql(sql_query)

In [68]:
result_df.show()

+------+----------+-----------+------------------+
|emp_no|first_name|  last_name|         dept_name|
+------+----------+-----------+------------------+
| 10001|    Georgi|    Facello|       Development|
| 10002|   Bezalel|     Simmel|             Sales|
| 10003|     Parto|    Bamford|        Production|
| 10004| Chirstian|    Koblick|        Production|
| 10005|   Kyoichi|   Maliniak|   Human Resources|
| 10006|    Anneke|    Preusig|       Development|
| 10007|   Tzvetan|  Zielinski|          Research|
| 10008|    Saniya|   Kalloufi|       Development|
| 10009|    Sumant|       Peac|Quality Management|
| 10010| Duangkaew|   Piveteau|Quality Management|
| 10010| Duangkaew|   Piveteau|        Production|
| 10011|      Mary|      Sluis|  Customer Service|
| 10012|  Patricio|  Bridgland|       Development|
| 10013| Eberhardt|     Terkki|   Human Resources|
| 10014|     Berni|      Genin|       Development|
| 10015|  Guoxiang|  Nooteboom|          Research|
| 10016|  Kazuhito|Cappelletti|

In [69]:
# result_df.write.format('jdbc').options(
#       url='jdbc:postgresql://localhost:5432/rahul',
#       driver='org.postgresql.Driver',
#       dbtable='result_emp',
#       user='postgres',
#       password='Meta@123').mode('append').save()


In [70]:
rahul_df=result_df.limit(10)

In [71]:
rahul_df.toPandas()

,emp_no,first_name,last_name,dept_name
0,10001,Georgi,Facello,Development
1,10002,Bezalel,Simmel,Sales
2,10003,Parto,Bamford,Production
3,10004,Chirstian,Koblick,Production
4,10005,Kyoichi,Maliniak,Human Resources
5,10006,Anneke,Preusig,Development
6,10007,Tzvetan,Zielinski,Research
7,10008,Saniya,Kalloufi,Development
8,10009,Sumant,Peac,Quality Management
9,10010,Duangkaew,Piveteau,Production


In [82]:
import mysql.connector as sql
import pandas as pd
db_connection = sql.connect(host='localhost', database='employees', user='root', password='Meta@123')
db_cursor = db_connection.cursor()
db_cursor.execute('SELECT * FROM result_df')
table_rows = db_cursor.fetchall()
df = pd.DataFrame(table_rows)

# db_connection = sql.connect(host='localhost', database='information_schema', user='root', password='Meta@123')
# db_cursor = db_connection.cursor()
# db_cursor.execute("select * from columns where table_schema = 'employees' order by table_name,ordinal_position")
# table_rows = db_cursor.fetchall()
# df = pd.DataFrame(table_rows)
print(df)


   0    1    2   3
0  1  abc  def  cs
1  1  abc  def  cs
2  1  abc  def  cs
3  1  abc  def  cs
4  1  abc  def  cs
5  1  abc  def  cs


In [ ]:
df.loc[df[]]

In [73]:
a = {"emp_no": 1, "first_name":"abc", "last_name": "def", "dept_name": 'cs'}
dfpandas = pd.DataFrame([a])
dfspark = spark.createDataFrame(dfpandas)

In [81]:
dfspark.write.format('jdbc').options(
    url='jdbc:mysql://localhost:3306/employees',
    driver='com.mysql.jdbc.Driver',
    dbtable='result_df',
    user='root',
    password='Meta@123').mode('append').save()

In [48]:
# rahul_df.write.csv('result.csv')